In [ ]:
#DEVELOPMET ROAD

#1 THEORY AND PREPARATION
#  go over the theory and explain what we are going to build prepare our data and environment

#TRAIN MODEL
#  train our model on the data we have downloaded

#CREATE THE GUI
#   Create a GUI to show our results in live live time



In [ ]:
#DEEP CONVULUTION

import numpy as np
import cv2
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten # are the layer we are using
from keras.layers import Conv2D #
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator #pre-processor imagedata generating



train_dir = 'data/train' #trainin model - directory
val_dir = 'data/test' #validate

# generate the data set from the image file in directory in c=scal 1 to 255 since in picles, its an array from 1 and 255
train_datagen = ImageDataGenerator(rescale=1./255) 
val_datagen = ImageDataGenerator(rescale=1./255)

#
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    
    #in ml, no. of training examples utilised in one iteration
    batch_size=64,
    
    # gray scale, black and white scale
    color_mode="grayscale",
    
    #since it has classsification of happy, sad,etc.
    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')


# creating model
# we are initialiding model(Sequential model)
# using keras we can use sequential and functional
# seq - creat layer by layer, but limit in depth, cannot share layers or multiple input or output
# fun - more flexible, find layers, find next layers, more complex
emotion_model = Sequential()

# adding layers

# special type of nural network, used for image recognision and classification
# convilution is simple applicatio of a flilter to an input that results in activation
# some input some thrushhold, if input meets thurshhold then there is activation
# repeated application of the same filter to an input results in map of optimation which is call feater map
# if input goes in , that repeats in certain way it forms map
# indicating location and strangs of a detected feter in the input, in our case images

# first layer 2D convulutional layer
# creats a convulutional kernel that is convolved with layer of input to form output
# 32-input shape
# hight, width-3,3
# represents pictures, also grayscale images
# relu- rectifier, which is an activation function used in nn(there is math)
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

#maxpooling2d - down sample our data,(to include less data, smaller subset of data)
# training on a disproportinaly low subset of the classic sample
# reverse - up waiting, adding a sample way to down sampleway class
emotion_model.add(MaxPooling2D(pool_size=(2,2)))

#droupout - randomly set input units to 0, with frequence rate at each step during the training time
# helps to prevent over fitting
# over fitting - a model that learned the noice instead of the signal that is considerd to be required signal forming
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

# have to flatten before turn all the neurons , 1024 in to 7(below values)
# 
emotion_model.add(Flatten())

# Dense - it is a hidden layer which has 1024 neurons/units
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))

# 7 neurons used cauese it tells exact number of possiblities i.e. 7 emotions
emotion_model.add(Dense(7, activation='softmax')) 



#compile model
# loss is degree of error
# model strive to minimize loss rather to increase accuracy
# optimizer adem - complex
# categorical cross - type the matrix how we specy the loss
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

# fit it
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64, # bentch size - 64
    epochs=10, # no. of iterations to be don
    validation_data=validation_generator,
    validation_steps=7178 // 64)

emotion_model.save_weights('model.h5')

In [ ]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk # we are inporting images using tkintre  
import os
import numpy as np
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import threading

emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

#loading the weights
emotion_model.load_weights('model.h5')
cv2.ocl.setUseOpenCL(False)


#creating dictionary for all emotion
emotion_dict = {0: "Angry", 1:"Disgusted", 2: "Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"}

#os library, path directiory name,absolut path of te emojies
cur_path = os.path.adirname(os.path.abspath(_file_))

emoji_dist={0:cur_path+"/emojis/angry.png", 1:cur_path+"/emojis/disgusted.png", 2:cur_path+"/emojis/fearful.png", 3:cur_path+"/emojis/happy.png", 4:cur_path+"/emojis/neutral.png", 5:cur_path+"/emojis/sad.png", 6:cur_path+"/emojis/surprised.png"}

global last_frame1
last_frame1 = np.zeroes((480, 640, 3), dtype=np.unit8)
global cap1
show_text=[0]
global frame_number



def show_subject():
    
    #open the video, path of video
    # 0 - web cam access
    cap1=cv2.VideoCapture(0)
    
    #if camera open or not
    if not cap1.isOpenened():
        print("Can't open the camera")
    
    global frame_number
    
    #max no. of frames
    length = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number+=1
    if frame_number >= length:
        exit()
        
    # seting the next fram to increment
    cap1.set(1, frame_number)
    
    # fram - it is an array
    # flage - check if we read something or not
    flag1, frame1 = cap1.read()
    
    #resizing
    frame1= cv2.resize(frame1,(600,500))
    
    # a box near the face
    # but open cv already has it
    bounding_box= cv2.CascadeClassifier()
    
    #convert image to gray
    #BGR to gray
    # why we do?
    # the memory usage, the bandwith, it will be fast processing time, less space
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for(x, y, w, h) in num_faces:
        # creating rectangle
        # recognising
        cv2.rectangle(frame1, (x,y-50), (x+w, y+h+10),(250,0,0), 2)
        roi_gray_frame = gray_frame[y:y+h, x:x+w]
        # we have already trained the images dor size 48
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1),0)
        prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        show_text[0]=maxindex
    if flag1 is None:
        print("Major error!")
    #getting our main window
    elif flag1:
        global last_frame1
        last_frame1= frame1.copy()
        pic= cv2.cvtColor(last_frame1, cv2.COLOR_BGR2GRB)
        #we have array from 0 to 255
        #generate image
        img= Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk=imgtk
        #since we are running in different thread we need to update our main thread
        # update main thread where our tkinter where gui is runnung
        root.update()
        
        #after - way to use tkinter to get around with blocking
        #after the time 10 sec is assed it will run even i fthe main function is running
        lmain.after(10, show_subject)
    if cv2.waitkey(1) & 0xFF == ord('q'):
        exit()

def show_avatar():
    frame2=cv2.imread(emoji_dist[show_text[0]])
    pic2=cv2.cvtColo(frame2,cv2.Color_BGR2RGB)
    img2=Image.fromarray(frame2)
    imgtk2= ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2=imgtk2
    lmain3.configure(text=emotion_dict[show_text[0]], font=('arial',45,'bold'))

    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, show_avatar)

# main file
if _name_ == '_main_' :
    #specifying frame number, lagter we will read through our file, increment frame ny frame
    frame_number= 0
    root=tk.TK()
    
    # creating lables that are going to contain our images or video
    lmain = tk.Label(master=root,padx=50,bd=10)
    lmain2 = tk.Label(master=root,bd=10)
    
    #for quiting
    lmain3 = tk.Label(master=root,bd=10,fg="#CDCDCD",bg='black')
    
    # packing and placing
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=250)
    lmain3.pack()
    lmain3.place(x=960,y=250)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=900,y=350)

    #specified title
    root.title("Photo To Emoji")
    
    #specified geomentry
    root.geometry("1400x900+100+10")
    root['bg']='black'
    
    #exit button on bottom
    exitButton = Button(root, text='Quit',fg='red',command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    
    #threading because main loop is blocking
    # when get to main loop everything is goin to run
    threading.Thread(target=show_subject).start()
    threading.Thread(target=show_avatar).start()
    root.mainloop()